In [102]:
import mibian
import pandas as pd
import numpy as np
import random

In [76]:
class Strategy(object):
    
    interestRate = 0.5
    
    def __init__(self, underPrice, Strike1, Strike2, Strike3, Strike4, days, impVol1, impVol2, impVol3, impVol4):
        self.underPrice = underPrice
        self.Strike1 = Strike1
        self.Strike2 = Strike2
        self.Strike3 = Strike3
        self.Strike4 = Strike4
        self.days = days
        self.impVol1 = impVol1
        self.impVol2 = impVol2
        self.impVol3 = impVol3
        self.impVol4 = impVol4

In [77]:
class volStrategy(Strategy):
    
    def __init__(self, stype):
        self.ftype = stype

    def straddle(underPrice, Strike, days, impVol):
        leg = mibian.BS([underPrice, Strike, Strategy.interestRate, days], volatility = impVol)
        return leg.callPrice + leg.putPrice

    def strangle(underPrice, Strike1, Strike2, days, impVol1, impVol2):
        leg1 = mibian.BS([underPrice, Strike1, Strategy.interestRate, days], volatility = impVol1)
        leg2 = mibian.BS([underPrice, Strike2, Strategy.interestRate, days], volatility = impVol2)
        if Strike1 >= Strike2:
            return leg1.callPrice + leg2.putPrice
        else:
            return leg1.putPrice + leg2.callPrice

In [90]:
class statStrategy(Strategy):
    
    def __init__(self, stype):
        self.ftype = stype
    
    def leverage():
        return self - (Strategy.Strike1 - Strategy.Strike2) 
                
        

In [79]:
class spread(statStrategy):
    
    def __init__(self, ftype):
        self.ftype = ftype
        
    def callSpread(underPrice, Strike1, Strike2, days, impVol1, impVol2):
        leg1 = mibian.BS([underPrice, Strike1, Strategy.interestRate, days], volatility = impVol1)
        leg2 = mibian.BS([underPrice, Strike2, Strategy.interestRate, days], volatility = impVol2)
        return abs(leg1.callPrice - leg2.callPrice)
    
    def putSpread(underPrice, Strike1, Strike2, days, impVol1, impVol2):
        leg1 = mibian.BS([underPrice, Strike1, Strategy.interestRate, days], volatility = impVol1)
        leg2 = mibian.BS([underPrice, Strike2, Strategy.interestRate, days], volatility = impVol2)
        return abs(leg1.putPrice - leg2.putPrice)

In [80]:
class fly(statStrategy):

    def __init__(self, ftype):
        self.ftype = ftype       
            
    def atmFly(underPrice, Strike1, Strike2, Strike3, days, impVol1, impVol2, impVol3):
        
        leg1 = mibian.BS([underPrice, Strike1, Strategy.interestRate, days], volatility = impVol1)
        leg2 = mibian.BS([underPrice, Strike2, Strategy.interestRate, days], volatility = impVol2)
        leg3 = mibian.BS([underPrice, Strike3, Strategy.interestRate, days], volatility = impVol3)
        
        if Strike1 >= Strike3:
            return leg1.callPrice + leg3.putPrice - (leg2.callPrice + leg2.putPrice)
        else:
            return leg1.putPrice + leg3.callPrice - (leg2.callPrice + leg2.putPrice)
        
    def callFly(underPrice, Strike1, Strike2, Strike3, days, impVol1, impVol2, impVol3):
    
        leg1 = mibian.BS([underPrice, Strike1, Strategy.interestRate, days], volatility = impVol1)
        leg2 = mibian.BS([underPrice, Strike2, Strategy.interestRate, days], volatility = impVol2)
        leg3 = mibian.BS([underPrice, Strike3, Strategy.interestRate, days], volatility = impVol3)
        
        return leg1.callPrice + leg3.callPrice - 2*leg2.callPrice
    
    def putFly(underPrice, Strike1, Strike2, Strike3, days, impVol1, impVol2, impVol3):
    
        leg1 = mibian.BS([underPrice, Strike1, Strategy.interestRate, days], volatility = impVol1)
        leg2 = mibian.BS([underPrice, Strike2, Strategy.interestRate, days], volatility = impVol2)
        leg3 = mibian.BS([underPrice, Strike3, Strategy.interestRate, days], volatility = impVol3)
        
        return leg1.putPrice + leg3.putPrice - 2*leg2.putPrice       
    

In [81]:
class condor(statStrategy):
    
    def __init__(self, ftype):
        self.ftype = ftype       
            
    def atmCondor(underPrice, Strike1, Strike2, Strike3, Strike4, 
             days, impVol1, impVol2, impVol3, impVol4):
        
        return volStrategy.strangle(underPrice, Strike1, Strike4, days, impVol1,
                                    impVol4) - volStrategy.strangle(underPrice, Strike2, Strike3, 
                                                                    days, impVol2, impVol3) 
        
    def callCondor(underPrice, Strike1, Strike2, Strike3, Strike4, 
                   days, impVol1, impVol2, impVol3, impVol4):
    
        leg1 = mibian.BS([underPrice, Strike1, Strategy.interestRate, days], volatility = impVol1)
        leg2 = mibian.BS([underPrice, Strike2, Strategy.interestRate, days], volatility = impVol2)
        leg3 = mibian.BS([underPrice, Strike3, Strategy.interestRate, days], volatility = impVol3)
        leg4 = mibian.BS([underPrice, Strike4, Strategy.interestRate, days], volatility = impVol4)
        
        return leg1.callPrice + leg3.callPrice - leg2.callPrice - leg4.callPrice
    
    def putCondor(underPrice, Strike1, Strike2, Strike3, Strike4, 
                  days, impVol1, impVol2, impVol3, impVol4):
    
        leg1 = mibian.BS([underPrice, Strike1, Strategy.interestRate, days], volatility = impVol1)
        leg2 = mibian.BS([underPrice, Strike2, Strategy.interestRate, days], volatility = impVol2)
        leg3 = mibian.BS([underPrice, Strike3, Strategy.interestRate, days], volatility = impVol3)
        leg4 = mibian.BS([underPrice, Strike4, Strategy.interestRate, days], volatility = impVol4)
        
        return leg1.putPrice + leg3.putPrice - leg2.putPrice - leg4.putPrice

In [197]:
price = 342.75
ndays = 30
nreps = 100
plong = 0.47
pshort = 0.49
maglong = 1.1
maglong_cents = price/100*maglong
magshort = 0.9
magshort_cents = -price/100*magshort
nmatrix = (ndays, nreps)
simarray = np.ones(nmatrix)
simfloat = np.apply_along_axis(lambda x: random.random(), 0, simarray)
simfloat.shape

(100,)

In [200]:
for i in range(ndays):    
    simarray[i] = np.apply_along_axis(lambda x: random.random(), 0, simarray[i])
simarray.shape

(30, 100)

In [220]:
for i in range(ndays):
    for n in range(ndays):
        if simarray[n, i] >= plong:
            simarray[n, i] = maglong_cents
        elif simarray[n, i] <= pshort:
            simarray[n, i] = magshort_cents
        else:
            simarray[n, i] = 0
    #simarray[i] = np.cumsum(simarray)

In [221]:
simprices = pd.DataFrame(simarray)

In [223]:
simprices.tail()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
25,3.77025,3.77025,3.77025,3.77025,3.77025,3.77025,3.77025,3.77025,3.77025,3.77025,...,343.09275,346.863,350.63325,354.4035,358.17375,361.944,365.71425,369.4845,373.25475,377.025
26,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,...,-280.71225,-283.797,-286.88175,-289.9665,-293.05125,-296.136,-299.22075,-302.3055,-305.39025,-308.475
27,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,...,-280.71225,-283.797,-286.88175,-289.9665,-293.05125,-296.136,-299.22075,-302.3055,-305.39025,-308.475
28,3.77025,3.77025,3.77025,3.77025,3.77025,3.77025,3.77025,3.77025,3.77025,3.77025,...,343.09275,346.863,350.63325,354.4035,358.17375,361.944,365.71425,369.4845,373.25475,377.025
29,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,-3.08475,...,-280.71225,-283.797,-286.88175,-289.9665,-293.05125,-296.136,-299.22075,-302.3055,-305.39025,-308.475
